<a href="https://colab.research.google.com/github/fjpa121197/DataScienceDM/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
#Credits to professor
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_arr[user_id][item_id]
                if not rating == 99.0:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 1000) == 0:
        print(mse)

In [97]:
#Read data
data_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv",header=None)

data_df.drop(data_df.columns[0],axis = 1, inplace = True)
print(data_df)
arr = data_df.values
#There are cells that have a 99, which means that they have a null value/not rated
rated_cells = np.where(arr != 99)

         1      2      3      4     5    ...    96     97     98     99     100
0      -7.82   8.79  -9.66  -8.16 -7.52  ...  99.00  -5.63  99.00  99.00  99.00
1       4.08  -0.29   6.36   4.37 -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
2      99.00  99.00  99.00  99.00  9.03  ...  99.00  99.00  99.00  99.00  99.00
3      99.00   8.35  99.00  99.00  1.80  ...  99.00  99.00  99.00  99.00  99.00
4       8.50   4.61  -4.17  -5.39  1.36  ...   1.55   3.11   6.55   1.80   1.60
...      ...    ...    ...    ...   ...  ...    ...    ...    ...    ...    ...
24978   0.44   7.43   9.08   2.33  3.20  ...   9.03   6.55   8.69   8.79   7.43
24979   9.13  -8.16   8.59   9.08  0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24980  99.00  99.00  99.00  99.00 -7.77  ...  99.00  99.00  99.00  99.00  99.00
24981  99.00  99.00  99.00  99.00 -9.71  ...  99.00  99.00  99.00  99.00  99.00
24982   2.43   2.67  -3.98   4.27 -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 100 columns]


In [86]:
print(arr)

[[-7.82  8.79 -9.66 ... 99.   99.   99.  ]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


In [98]:
new_arr, idx = replace(arr,0.1)
print(new_arr)

[[-7.82  8.79 -9.66 ... 99.   99.   99.  ]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 99.   ... 99.   99.   99.  ]]


In [99]:
print(idx)

(array([  672, 24140, 18848, ..., 14875, 11082, 24507]), array([24, 13, 52, ..., 86, 51, 51]))


In [0]:
#Start of Latent Factor Modelling

n_latent_factors = 3

#Random values
latent_user_preferences = np.random.random((new_arr.shape[0], n_latent_factors))
latent_item_features = np.random.random((new_arr.shape[1], n_latent_factors))

In [120]:
sgd()

24.59717756667817
22.988005644724133
20.899147627735154
18.491972390486087
17.73496261494542
17.603924924806893
17.535394756235164
17.41980346274494
17.28716139046625
17.256379893067297


In [121]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-3.58370306, -5.12036302, -5.92562258, ..., -3.82144375,
        -6.06110151, -3.84940558],
       [ 2.89330651,  4.66387742,  4.29055974, ...,  3.91303375,
         2.80630735,  4.22913633],
       [ 7.08456034,  7.79072628,  8.06220706, ...,  8.74404145,
         8.54324893,  9.32040484],
       ...,
       [ 0.25304983,  2.890389  ,  1.11636702, ...,  1.68700018,
        -2.9146416 ,  2.13943359],
       [-1.84108757, -2.57245959, -3.24831489, ..., -1.74805095,
        -3.66767532, -1.683559  ],
       [ 3.83306048,  3.93865451,  3.92415174, ...,  4.87595399,
         4.20407542,  5.25107909]])

In [0]:
values = [zip(new_arr[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data_df.columns

In [125]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -3.583703055010273)","(8.79, -5.120363018025969)","(-9.66, -5.9256225846935875)","(-8.16, -9.97509446525483)","(-7.52, -3.23919218897141)","(-8.5, -1.7968416350478034)","(-9.85, -5.4842875549248244)","(99.0, -4.71314577599954)","(-8.98, -8.28129114670726)","(-4.76, -4.147936453211377)","(-8.5, -2.227635829438893)","(-6.75, -1.7560677233068536)","(-7.18, -7.431439672823966)","(8.45, -1.819820806408965)","(99.0, -8.059812203151335)","(-7.52, -10.634907368704205)","(-7.43, -5.58496426588922)","(-9.81, -5.713695207082354)","(-9.85, -3.8929681881304625)","(-9.85, -7.273830481988602)","(-9.37, -0.11528007782643858)","(1.5, -4.8326405980052325)","(-4.37, -6.314053514180973)","(-9.81, -10.999028684570025)","(-8.5, -5.3001858834816735)","(99.0, -2.333480216409925)","(7.82, 3.11802453002881)","(2.86, -1.402482677716774)","(9.13, 2.9819450728599968)","(-7.43, -8.227472701204274)","(2.14, 0.530407772102796)","(-4.08, 3.5865302180699086)","(-9.08, -10.581931830735018)","(7.82, -3.476850212771354)","(5.05, 2.781905843564069)","(4.95, 3.7081045482139907)","(-9.17, -10.029056306542325)","(-8.4, -4.182703211267033)","(-8.4, -3.097385386103174)","(-8.4, -4.423137504957053)",...,"(8.59, 0.6247869392786389)","(3.59, 2.405594733831902)","(-6.84, -6.34819484836945)","(-9.03, -9.515995708686996)","(2.82, 0.8928471073668155)","(-1.36, 1.3809614809235562)","(-9.08, -9.87772892219766)","(8.3, 1.9891652589713207)","(99.0, 1.453035160969375)","(-4.81, -5.038430198453255)","(99.0, -7.381232802833735)","(99.0, 1.8959274717143972)","(99.0, -2.797920745176573)","(99.0, -9.117199036370982)","(99.0, -6.554372443260476)","(99.0, 0.7226010331521359)","(99.0, -4.20143707461189)","(-9.42, -2.262391124945077)","(99.0, -6.213488627608149)","(99.0, -4.010482615408501)","(99.0, -1.4693360900745054)","(-7.72, -3.998659442052543)","(99.0, -0.812640011664976)","(99.0, -4.151643228685291)","(99.0, -4.947763669352034)","(99.0, -5.019160675679956)","(99.0, -0.8794550308090108)","(99.0, -0.28028762809643026)","(99.0, 3.983717018036599)","(99.0, -4.640059308786497)","(2.82, -0.34264959194899536)","(99.0, -2.574682015273549)","(99.0, 1.0351854827559672)","(99.0, -3.6643169950495484)","(99.0, -2.956480422325457)","(99.0, -2.0737386713969563)","(99.0, -1.5428372871189304)","(99.0, -3.8214437544363875)","(99.0, -6.061101508795846)","(99.0, -3.849405583689203)"
1,"(4.08, 2.893306509935147)","(-0.29, 4.663877415875452)","(6.36, 4.290559738691808)","(4.37, 5.947578552375452)","(99.0, 2.936451169692241)","(-9.66, 2.334359751433237)","(-0.73, 3.477824024394671)","(-5.34, 2.283724620973839)","(99.0, 3.907776023793335)","(9.22, 2.8549750420603015)","(6.75, 3.1136354373423174)","(8.64, 3.052981023921099)","(4.42, 3.7615652645806095)","(7.43, 3.3456058914444275)","(4.56, 3.5875822061705684)","(-0.97, 3.626036945350539)","(4.66, 3.6049753612505286)","(-0.68, 2.182596251085459)","(3.3, 1.9316903552868743)","(-1.21, 3.2751835283020108)","(99.0, 2.9073616163458955)","(8.64, 4.257502285407712)","(8.35, 3.5907753226169463)","(9.17, 5.146586202911124)","(0.05, 4.5893204303594874)","(7.57, 3.560401657260323)","(4.71, 0.7080601211446459)","(0.87, 1.4570820152437116)","(-0.39, 1.1884560931536838)","(6.99, 4.606252386894806)","(6.5, 2.257988516733991)","(-0.92, -0.15648741191889676)","(99.0, 5.990427769559656)","(9.03, 3.985482201121436)","(-1.8, 0.742626266392815)","(0.73, 0.1257936997534967)","(7.09, 5.009980512249271)","(3.4, 3.0789215815694244)","(99.0, 3.7437731825313585)","(7.91, 4.058260994327537)",...,"(-6.7, 1.604522379617988)","(-3.35, -0.08309370982225665)","(-9.03, 3.4007913318719307)","(4.47, 4.962134803495538)","(4.08, 2.1615427417143773)","(-3.83, 0.9071094421559152)","(8.74, 4.7306493952657975)","(1.12, 0.07102632574603623)","(0.78, 1.27939